# <center> Fetching data from Riot API server </center>

### Pulls recent match details from players listed in Tier/Division ranking
#### Read more about the LOL API [here](https://developer.riotgames.com/docs/portal)!

#### The notebook goes through the following steps:

1. Pulling summoner names from leagues and saving table to data folder
2. Pulling accountid for summoners names to be able to get match details
3. Pulling match history for each player
4. Pulling game details for each matchID

In [58]:
from JinxData import LOL_API_Connection as LOL
import pandas as pd
from time import sleep
from time import time
from datetime import datetime
from ratelimit import limits, sleep_and_retry
import json
import os

In [59]:
ROOT_DIR = '.'
DATA_DIR = 'data'

### Insert LOL API key and Region here:

In [60]:
API_key = 'RGAPI-e502b975-1fbe-4499-9b60-c67fb91a9932'
region = 'eun1'

In [61]:
# Setting up connection
api = LOL(API_key, region)

**The rate limit for a personal keys is limited:**
- 20 requests every 1 second
- 100 requests every 2 minutes

**Below you can adjust the number of *calls* within a specific *period* (*s*):**

In [62]:
calls = 100
period = 123
@sleep_and_retry
@limits(calls=calls, period=period)
def get_response(conn_type, value_list, i, **filters):
    return conn_type(value_list[i], **filters)

def get_records(conn_type, value_list, verbose = 100, sleeptime = 1, **filters):
    '''Pull records from the list with a specific data collector.
    Returns a list with the requested records as dicts. Accepts filters as **kwargs'''
    newdict = {} 
    errors = []
    i = 0
    while i < len(value_list):

        newdict[value_list[i]] = get_response(conn_type, value_list, i, **filters)
                
        if 'status' in newdict[value_list[i]]:                                              # error handling
            status = newdict[value_list[i]]['status']
            print('{} Error: {}'.format(datetime.now().strftime('%H:%M:%S'), str(status) ) )
            errors.append([time(), status['status_code']] )
            maxerror = 25
            del newdict[value_list[i]]
            if len(errors) >= maxerror and errors[-1][0] - errors[-maxerror][0] < 2*60:     # break loop if too many errors within an interval
                print(errors)
                print('Session interupted due to excessive errors')
                break
            elif errors[-1][1] == 404:
                i +=1
            i -= 1
            sleep(2)
        else:
            sleep(sleeptime)
            #counter = len(value_list) - i -1
            if verbose != 0 and i % verbose == 0:
                print('{0}: Loaded: {1} {2:.0f}% | Time remaining: {3:0.1f} min'.format(datetime.now().strftime('%H:%M:%S'), i, int((i+1)/len(value_list)*100), (len(value_list) - i -1)*1.2/60 ) )
        i += 1
    print('---FINISHED---')
    return newdict

--------------------------
### 1. Pulling summoner names from leagues and saving table to data folder

In [63]:
divisions = ['I','II','III','IV']
tiers = ['DIAMOND', 'PLATINUM', 'GOLD', 'SILVER', 'BRONZE', 'IRON']

Number of request: 4 (divisions) x 6 (tiers) = 24

In [64]:
league_names = {}
for tier in tiers:
    for div in divisions:
        league_names[tier + '-' + div] = api.GetSummonerNames(division = div, tier = tier)

KeyboardInterrupt: 

In [ ]:
summoner_list = pd.DataFrame()
for key in league_names:
    summoner_list = summoner_list.append( pd.DataFrame( league_names[key]), ignore_index= True )

In [ ]:
summoner_list.head(3)

In [ ]:
#Saving to csv
summoner_list.to_csv(os.path.join(ROOT_DIR, DATA_DIR, 'summoner_list.csv'))

-----------------------
## 2. Pulling accountid for summoners names to be able to get match details

In [ ]:
#Loading in saved csv file
summoner_list = pd.read_csv(os.path.join(ROOT_DIR, DATA_DIR, 'summoner_list.csv'))
summonerName = list(summoner_list.summonerName)

#### Requesting summonerIds name by name. This may take up to 2 hours beacuase of the rate limitations.

In [ ]:
sumIDs = get_records(api.GetSummonerID, summonerName[0:10], verbose = 1)

In [ ]:
id_list = pd.DataFrame( [sumIDs[key] for key in sumIDs.keys()] )
id_list.head()

In [ ]:
#Saving to csv
id_list.to_csv(os.path.join(ROOT_DIR, DATA_DIR, 'summoner_id_details.csv'))

----------------------
## 3. Pulling match history for each player

In [ ]:
#Loading in saved csv file
id_list = pd.read_csv(os.path.join(ROOT_DIR, DATA_DIR, 'summoner_id_details.csv'))
accountId = list(id_list['accountId'])

**This may take several hours**

In [ ]:
match_hist = get_records(api.GetMatchHist_full, accountId[0:10], verbose=1, sleeptime=0) 

In [ ]:
match_list = pd.DataFrame()
for key, value in match_hist.items():
    id_hist = pd.DataFrame(value)
    id_hist['accountId'] = key
    match_list = match_list.append(id_hist)
match_list.reset_index(inplace = True)
match_list.head()

In [ ]:
#Saving to csv
match_list.to_csv(os.path.join(ROOT_DIR, DATA_DIR, 'match_history.csv'))

In [ ]:
match_list.shape

--------------------
## 4. Pulling game details for each matchID
This part creates a new folder in data dir then saves each player's match detail into a separate json file 

In [ ]:
#Loading in saved csv file
match_list = pd.read_csv(os.path.join(ROOT_DIR, DATA_DIR, 'match_history.csv'))

In [ ]:
gameids = match_list[['gameId', 'accountId']]
gameids.head(2)

In [ ]:
path = os.path.join(ROOT_DIR, DATA_DIR, 'match_details_by_summoners')
os.makedirs(path, exist_ok=True)

In [ ]:
def save_file(path, name, data):
    filename = os.path.join(path, name + '.json')
    jsonfile = json.dumps(data)
    file = open(filename, 'w')
    file.write(jsonfile)
    file.close()

In [ ]:
# Removing items that were already downloaded (already in the downloaded directory)
dir_content = [s.replace('.json','') for s in os.listdir(path)]

full_accountid_list = list(gameids['accountId'].unique()[0:4])  # Now only pulls 5 players match details

accountid_list = [s for s in full_accountid_list if s not in dir_content]

In [ ]:
for id in accountid_list:
    games = list(gameids[ gameids.accountId == id]['gameId'])
    
    # Get all game details in the games list
    game_detail_dict = get_records(api.GetMatchDetails, games, verbose=8 , sleeptime=0.3)
     
    save_file(path, id, game_detail_dict)
    print('{}% of files saved!'.format( int((accountid_list.index(id)+1)/len(accountid_list)*100)) )

----------------------
# Merging all match detail files to masterdata

In [ ]:
from src.ProgressBar import update_progress

In [5]:
path = os.path.join(ROOT_DIR, DATA_DIR, 'match_details_by_summoners')

In [6]:
def game_participants_to_df(data, gameid):
    '''Merging player details and match details in one file'''
    players_in_game = pd.DataFrame()
    n_of_participants = len(data[gameid]['participants'])
    for i in range(n_of_participants): # going through the participants and adding other info 
        # 1. Creating participant row with partIdentities
        part_id = data[gameid]['participantIdentities'][i]['participantId']
        participant_row = pd.DataFrame(data[gameid]['participantIdentities'][i]['player'], index=[part_id])

        # 2. Adding games stasts to the row
        stats = pd.DataFrame(data[gameid]['participants'][i]['stats'], index = [part_id] ).rename(columns= {'inhibitorKills': 'inhibitorKills_participant'})
        lane = pd.DataFrame(data[gameid]['participants'][i]['timeline'], index = [part_id] )[['role', 'lane']]
        stats = stats.join(lane)
        newkeys = [key for key in data[gameid]['participants'][i].keys() if key not in ['stats', 'timeline', 'participantId'] ]
        for key in newkeys:
            stats[key] = data[gameid]['participants'][i][key]
        participant_row = participant_row.join(stats)

        # 3. Adding teams stats
        teamstat = pd.DataFrame(data[gameid]['teams']).drop('win', axis = 1).set_index('teamId')
        participant_row = participant_row.merge(teamstat, on = 'teamId')

        # 4. Add row to the total list
        players_in_game = players_in_game.append( participant_row )

        # 5. Adding game desc details to the game dataframe
        newkeys = [key for key in data[gameid].keys() if key not in ['participants', 'participantIdentities', 'teams'] ]
        for key in newkeys:
            players_in_game[key] = data[gameid][key]
    players_in_game.reset_index(drop = True, inplace = True)
    return players_in_game

In [12]:
# Loops through the match_details_by_summoners dir files and 

FolderItems = os.listdir(path)
games = pd.DataFrame()
for file in FolderItems:
    json_file = open(os.path.join(path, file))
    data = json.load(json_file)
    for gameid in data:
        games = games.append(game_participants_to_df(data, gameid))
        games.reset_index(drop = True)
    update_progress((FolderItems.index(file)+1)/len(FolderItems))

AttributeError: 'list' object has no attribute 'astype'

In [47]:
games.to_csv(os.path.join(ROOT_DIR, DATA_DIR, 'match_detail_master.csv')

In [46]:
games.shape

(19882, 142)